## Load and parse the data

In [ ]:
biomarkers = load_data(filename='Biomarkers_Cells.csv')
biomarkers.head()

In [ ]:
clean_metadata(biomarkers)
biomarkers.head(3)

In [ ]:
biomarkers = move_column(biomarkers, 'label')
biomarkers.head(3)

In [ ]:
print(biomarkers.shape)
list(biomarkers.columns)

In [ ]:
drop_cols = ['Location_Center_X', 'Location_Center_Y', 'Location_Center_Z',
             'Number_Object_Number', 'Parent_Nuclei']
biomarkers.drop(drop_cols, axis=1, inplace=True)

## Missing values?

In [ ]:
# Are there any missing values?
print(biomarkers.isnull().sum().sum())

## Descriptive statistics

In [ ]:
stats = biomarkers.describe()
stats

## Drop the features with zero variance

In [ ]:
biomarkers.drop(stats.columns[stats.loc['std', :] == 0], axis=1, inplace=True)

In [ ]:
print("New shape is: {}".format(biomarkers.shape))
biomarkers.dtypes.value_counts()

## Create dataframes for Keratin and Vimentin

In [ ]:
ker_cols = [col for col in biomarkers.columns if 'OrigKer' in col]
print(len(ker_cols))

In [ ]:
vim_cols = [col for col in biomarkers.columns if 'OrigVim' in col]
print(len(vim_cols))

In [ ]:
keratin = biomarkers.copy()
keratin.drop(vim_cols, axis=1, inplace=True)

print(keratin.shape)
keratin.head(3)

In [ ]:
vimentin = biomarkers.copy()
vimentin.drop(ker_cols, axis=1, inplace=True)

print(vimentin.shape)
vimentin.head(3)

## Correlations between features

### Keratin

In [ ]:
keratin_ = keratin.copy()
keratin_['stiffness_num'] = pd.to_numeric(keratin_.stiffness)

In [ ]:
correlations = keratin_.corr();
correlations.stiffness_num

In [ ]:
plt.rcParams['figure.figsize'] = (12, 10)

sns.heatmap(correlations, cmap=sns.color_palette("PRGn_r", 100), vmin=-1, vmax=1);

plt.savefig('../results/Keratin Correlations.png', bbox_inches='tight', dpi=300);

### Vimentin

In [ ]:
vimentin_ = vimentin.copy()
vimentin_['stiffness_num'] = pd.to_numeric(vimentin_.stiffness)

In [ ]:
correlations = vimentin_.corr();
correlations.stiffness_num

In [ ]:
plt.rcParams['figure.figsize'] = (12, 10)

sns.heatmap(correlations, cmap=sns.color_palette("PRGn_r", 100), vmin=-1, vmax=1);

plt.savefig('../results/Vimentin Correlations.png', bbox_inches='tight', dpi=300);

## Intensity by stiffness

### Keratin

In [ ]:
keratin['total_px'] = keratin['Intensity_IntegratedIntensity_OrigKer'] / keratin['Intensity_MeanIntensity_OrigKer']
keratin['edge_px'] = keratin['Intensity_IntegratedIntensityEdge_OrigKer'] / keratin['Intensity_MeanIntensityEdge_OrigKer']

In [ ]:
keratin.head(3)

In [ ]:
keratin['IntegratedIntensityInner'] = keratin['Intensity_IntegratedIntensity_OrigKer'] - keratin['Intensity_IntegratedIntensityEdge_OrigKer']

keratin['MeanIntensityInner'] = keratin['IntegratedIntensityInner'] / (keratin['total_px'] - keratin['edge_px'])

In [ ]:
intensity_cols = ['Intensity_IntegratedIntensity_OrigKer', 
                  'IntegratedIntensityInner', 
                  'Intensity_IntegratedIntensityEdge_OrigKer']

fig, ax = plt.subplots()
keratin.groupby('stiffness')[intensity_cols].mean().plot(kind='bar', 
                                                         title="Integrated intensity of Cytokeratin", 
                                                         fontsize=14,
                                                         ax=ax);
ax.legend(["Integrated Intensity", "Integrated Inner Intensity", "Integrated Edge Intensity"], fontsize=14);

In [ ]:
intensity_cols = ['Intensity_MeanIntensity_OrigKer', 
                  'MeanIntensityInner', 
                  'Intensity_MeanIntensityEdge_OrigKer']

fig, ax = plt.subplots()
keratin.groupby('stiffness')[intensity_cols].mean().plot(kind='bar', 
                                                         title="Mean intensity of Cytokeratin", 
                                                         fontsize=14,
                                                         ax=ax);
ax.legend(["Mean Intensity", "Mean Inner Intensity", "Mean Edge Intensity"], fontsize=14);

plt.savefig('../results/Cytokeratin Mean Intensity.png', bbox_inches='tight', dpi=300);

### Vimentin

In [ ]:
vimentin['total_px'] = vimentin['Intensity_IntegratedIntensity_OrigVim'] / vimentin['Intensity_MeanIntensity_OrigVim']
vimentin['edge_px'] = vimentin['Intensity_IntegratedIntensityEdge_OrigVim'] / vimentin['Intensity_MeanIntensityEdge_OrigVim']

In [ ]:
vimentin.head(3)

In [ ]:
vimentin['IntegratedIntensityInner'] = vimentin['Intensity_IntegratedIntensity_OrigVim'] - vimentin['Intensity_IntegratedIntensityEdge_OrigVim']

vimentin['MeanIntensityInner'] = vimentin['IntegratedIntensityInner'] / (vimentin['total_px'] - vimentin['edge_px'])

In [ ]:
intensity_cols = ['Intensity_IntegratedIntensity_OrigVim', 
                  'IntegratedIntensityInner', 
                  'Intensity_IntegratedIntensityEdge_OrigVim']

fig, ax = plt.subplots()
vimentin.groupby('stiffness')[intensity_cols].mean().plot(kind='bar', 
                                                         title="Integrated intensity of Vimentin", 
                                                         fontsize=14,
                                                         ax=ax);
ax.legend(["Integrated Intensity", "Integrated Inner Intensity", "Integrated Edge Intensity"], fontsize=14);

In [ ]:
intensity_cols = ['Intensity_MeanIntensity_OrigVim', 
                  'MeanIntensityInner', 
                  'Intensity_MeanIntensityEdge_OrigVim']

fig, ax = plt.subplots()
vimentin.groupby('stiffness')[intensity_cols].mean().plot(kind='bar', 
                                                         title="Mean intensity of Vimentin", 
                                                         fontsize=14,
                                                         ax=ax);
ax.legend(["Mean Intensity", "Mean Inner Intensity", "Mean Edge Intensity"], fontsize=14);

plt.savefig('../results/Vimentin Mean Intensity.png', bbox_inches='tight', dpi=300);

## Normalise signal and compare

In [ ]:
max_ker = keratin[['Intensity_MeanIntensity_OrigKer']].max()
max_ker

In [ ]:
keratin[['Intensity_MeanIntensity_OrigKer']].head()